In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

/Users/hong/venvs/nlp_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('spam.csv', encoding='latin1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [4]:
df = df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df['v1'].value_counts() # stratify should be applied

v1
ham     4825
spam     747
Name: count, dtype: int64

In [6]:
X = df['v2'].values
y = df['v1'].values

In [7]:
X

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [8]:
y

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [36]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y

array([0, 0, 1, ..., 0, 0, 0])

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3900,), (1672,), (3900,), (1672,))

In [55]:
# max_words = 1000

# tokenizer = Tokenizer(num_words=max_words)
tokenizer = Tokenizer() # has function to remove 
tokenizer.fit_on_texts(X_train) 

In [56]:
print(tokenizer.word_index, "\n")
print(tokenizer.word_counts, "\n")
print(tokenizer.word_docs, "\n")
print(tokenizer.document_count, "\n")

{'i': 1, 'to': 2, 'you': 3, 'a': 4, 'the': 5, 'u': 6, 'and': 7, 'is': 8, 'in': 9, 'me': 10, 'my': 11, 'your': 12, 'for': 13, 'it': 14, 'of': 15, 'have': 16, 'call': 17, 'on': 18, 'that': 19, 'now': 20, '2': 21, 'are': 22, 'so': 23, 'but': 24, 'not': 25, 'do': 26, 'can': 27, "i'm": 28, 'or': 29, 'at': 30, 'ur': 31, 'get': 32, 'if': 33, 'be': 34, 'with': 35, 'just': 36, 'will': 37, 'we': 38, 'this': 39, 'no': 40, 'gt': 41, 'lt': 42, '4': 43, 'up': 44, 'when': 45, 'free': 46, 'go': 47, 'from': 48, 'how': 49, 'ok': 50, 'know': 51, 'all': 52, 'out': 53, 'what': 54, 'got': 55, 'good': 56, 'then': 57, 'time': 58, 'like': 59, 'am': 60, 'was': 61, 'its': 62, 'come': 63, 'there': 64, 'want': 65, 'only': 66, 'day': 67, 'send': 68, 'text': 69, 'love': 70, 'he': 71, 'txt': 72, 'as': 73, 'by': 74, 'going': 75, 'about': 76, 'one': 77, 'home': 78, 'stop': 79, "i'll": 80, 'da': 81, 'lor': 82, 'sorry': 83, 'our': 84, 'need': 85, 'still': 86, 'n': 87, 'r': 88, 'see': 89, 'reply': 90, 'back': 91, "don't":

In [ ]:
sequences = tokenizer.texts_to_sequences(X_train)
print(X_train, "\n")
print(sequences, "\n")

['Goal! Arsenal 4 (Henry, 7 v Liverpool 2 Henry scores with a simple shot from 6 yards from a pass by Bergkamp to give Arsenal a 2 goal margin after 78 mins.'
 "I dont. Can you send it to me. Plus how's mode."
 "Aah bless! How's your arm?" ...
 "Sure, if I get an acknowledgement from you that it's astoundingly tactless and generally faggy to demand a blood oath fo"
 'Nothing but we jus tot u would ask cos u ba gua... But we went mt faber yest... Yest jus went out already mah so today not going out... Jus call lor...'
 'Bull. Your plan was to go floating off to IKEA with me without a care in the world. So i have to live with your mess another day.'] 

[[1827, 2320, 43, 2321, 293, 231, 1828, 21, 2321, 2322, 35, 4, 669, 2323, 48, 222, 3507, 48, 4, 792, 74, 3508, 2, 134, 2320, 4, 21, 1827, 3509, 166, 3510, 263], [1, 94, 27, 3, 68, 14, 2, 10, 443, 421, 934], [3511, 1829, 421, 12, 2324], [112, 95, 1, 37, 63, 142], [328, 50], [71, 617, 2, 79, 75, 2, 384, 7, 149, 35, 5, 670, 2325], [26, 3, 65,

In [58]:
max_len = max([len(sequence) for sequence in sequences])
print(max_len)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

162


In [59]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

7410

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
    tf.keras.layers.LSTM(32), # return_sequences 각 층에서 결과값을 출력할지(True, 최종 3차원), 최종 결과값만 출력할지(False, 최종 1차원)
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [61]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
history = model.fit(sequences_matrix, y_train, batch_size=64, epochs=10, validation_split=0.2) # validation_split means X_train will be separated to valid.

Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8347 - loss: 0.4638 - val_accuracy: 0.9590 - val_loss: 0.1434
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9732 - loss: 0.1172 - val_accuracy: 0.9885 - val_loss: 0.0639
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9897 - loss: 0.0546 - val_accuracy: 0.9859 - val_loss: 0.0504
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9951 - loss: 0.0285 - val_accuracy: 0.9885 - val_loss: 0.0459
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9968 - loss: 0.0170 - val_accuracy: 0.9923 - val_loss: 0.0395
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9995 - loss: 0.0072 - val_accuracy: 0.9897 - val_loss: 0.0414
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 0.0045 - val_accuracy: 0.9923 - val_loss: 0.0392
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.9923 - v

In [64]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

# 이진분류 예측 및 평가
sequences_test = tokenizer.texts_to_sequences(X_test)
padded_sequences_test = sequence.pad_sequences(sequences_test)
y_pred = model.predict(padded_sequences_test)

# y_pred와 y_test의 shape 확인
print(f"y_pred shape: {y_pred.shape}")
print(f"y_test shape: {y_test.shape}")

# y_pred 처리 (sigmoid 출력이므로 0.5 threshold 사용)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

# y_test는 이미 binary 형태이므로 그대로 사용
y_test_classes = y_test  # 이미 [0, 1, 0, 1, ...] 형태

# shape 확인
print(f"y_pred_classes shape: {y_pred_classes.shape}")
print(f"y_test_classes shape: {y_test_classes.shape}")

# 평가
accuracy = accuracy_score(y_test_classes, y_pred_classes)
report = classification_report(y_test_classes, y_pred_classes, target_names=['ham', 'spam'])

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
y_pred shape: (1672, 1)
y_test shape: (1672,)
y_pred_classes shape: (1672,)
y_test_classes shape: (1672,)
Accuracy: 0.9880
Classification Report:
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1448
        spam       0.98      0.93      0.95       224

    accuracy                           0.99      1672
   macro avg       0.98      0.96      0.97      1672
weighted avg       0.99      0.99      0.99      1672

